# Hente filer fra Oracle
- Det finnes flere oppskrifter for å hente data fra andre Databaser på Bitbucket

## Leser inn nødvendige pakker

In [ ]:
#Trengs for eksempel 1
import pandas as pd
import cx_Oracle as cx
import getpass

#Tillegg for eksempel 2. Benyttes ikke i spørringen, kun under "ordner data".
import numpy as np

## Eksempel 1 - ISEE
Henter Driller-skjema

### Åpner forbindelse til Oracle

In [ ]:
username = "mtk"
db = "DB1P"
con = cx.connect(username,getpass.getpass(),db)

### Henter data
- Definerer delregisternummer
- Definerer skjema

In [ ]:
dnr = 425221
query = """
select * 
from dynarev.vw_funk_KOSTRA0A 
where delreg_nr = :dnr
"""
kostra0a = pd.read_sql(query,con=con,params={"dnr": str(dnr)})

### Ordner data
- Dette er ikke nødvendig for spørringen, men utføres for å vise. 
- Kan sikkert avgrenses direkte i spørringen i stedet.

In [ ]:
kostra0a = kostra0a[['PERIODE', 'REGION', 'KONTOKLASSE', 'FUNKSJON_KAPITTEL', 'ART_SEKTOR', 'BELOP', 'BYDEL']]

### Lagrer tabellen 
- Lagres i parquet format *(anbefales fra IT, usikker på hvorfor)*
- For å lagre i et annet format: bytt ut 'parquet' i både funkjson og filnavn med noe annet, f.eks. csv
- Fil lagres i samme mappe som denne arbeidsboken med mindre annen filsti defineres.

In [ ]:
kostra0a.to_parquet("kostra0a.parquet")

### Stenger forbindelsen til Oracle

In [ ]:
con.close()

## Eksempel 2 - DB1P
- Henter skjema fra dsbbase

### Åpner forbindelse til Oracle
Samme som i eksempel 1, men skrevet på en litt annen måte.  
Brukes etter preferanse. F.eks. om du skal benytte arbeidsboken alene, eller om den skal deles. 

In [ ]:
bruker = input('Angi bruker: ') or getpass.getuser()

In [ ]:
conn = cx.connect(bruker + "/" + getpass.getpass(prompt='Angi oracle-passord for bruker ' + bruker + ':')+"@DB1P")

### Henter data
Data hentes ved en SQL spørring.  
Skjema må defineres. I eksempelet henstes data fra ssb_foretak.  
Avgrenser data i spørringen ved hjelp av where-setningen.

In [ ]:
query = """
 select *
 from dsbbase.ssb_foretak
 where (org_form = 'KOMM' or org_form = 'FYLK' or org_form = 'IKS') and  (statuskode = 'B' or statuskode = 'S')
 """

komm = pd.read_sql_query(query, conn)

### Ordner data
Ikke nødvendig for spørringen, men inkluderes som et eksempel på hva som kan gjøres. 
1. Begrenser kolonner
2. Lager en ny kolonne basert på en eksisterende
3. Endrer navn på kolonnene
4. Skriver ut en visning av tabellen

In [ ]:
komm = komm[['ORGNR','ORG_FORM','STATUSKODE', 'NAVN', 'F_KOMMUNENR']]

komm['sortering']= np.where(komm['ORG_FORM']== 'KOMM', 1, 2)

komm.rename(columns={'ORGNR':'orgnr_eier','NAVN':'navn_eier', 'F_KOMMUNENR':'kommnr_eier', 'STATUSKODE':'status_eier', 'ORG_FORM':'orgform_eier'}, inplace = True)

komm

### Stenger forbindelsen til Oracle

In [ ]:
conn.close()

---
## Oppskrift ISEE, tekstlig:

### Python

import pandas as pd  
import cx_Oracle as cx  
import getpass  

username = **"Sett_inn_brukernavn"**  
db = "DB1P"  
con = cx.connect(username,getpass.getpass(),db)  
dnr = **Sett_inn_delregisternummer**  
query = "SELECT * FROM dynarev.vw_funk_**Sett_inn_skjema** where delreg_nr = :dnr"  
mine_data = pd.read_sql(query,con=con,params={"dnr": str(dnr)})  
con.close()  

### R

library(getPass)  
library(ROracle)  

drv <- dbDriver("Oracle")  
con <- dbConnect(drv, user = **"Sett_inn_brukernavn"**, password =getPass(), dbname = "DB1P")  
Mine_data <- dbGetQuery(con, "SELECT * FROM dynarev.VW_FUNK_**Sett_inn_skjema** WHERE DELREG_NR=**Sett_inn_delregisternummer**")
---